In [2]:
import sklearn
import pandas as pd
import numpy as np

In [27]:
data_folder = '~/instacart_data/'
aisles = pd.read_csv(data_folder + 'aisles.csv')
departments = pd.read_csv(data_folder + 'departments.csv')

print('loading products')
products = pd.read_csv(data_folder+ 'products.csv', dtype={
        'product_id': np.uint16,
        'order_id': np.int32,
        'aisle_id': np.uint8,
        'department_id': np.uint8},
        usecols=['product_id', 'aisle_id', 'department_id'])
    
print('loading prior')
priors = pd.read_csv(data_folder + 'order_products__prior.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8})

print('loading train')
train = pd.read_csv(data_folder + 'order_products__train.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8})

print('loading orders')
# replace orders.csv with orders_small.csv for faster loading time
orders = pd.read_csv(data_folder + 'orders.csv', dtype={
        'order_id': np.int32,
        'user_id': np.int32,
        'eval_set': 'category',
        'order_number': np.int16,
        'order_dow': np.int8,
        'order_hour_of_day': np.int8,
        'days_since_prior_order': np.float32})

loading products
loading prior
loading train
loading orders


orders_full was made in `make_orders_full.ipynb`

Notice that we have product id, product name, aisle id, and deparment id. The product name could be useful in the future if we wanted to do some sort of word embeddings with these names, for example we could group all the 'cookies' together or all the 'teas'. The aisle id would also be used as a way to group certain food types together since the grocery stores group similar items together. The department id, as we will see in the next cell, is another grouping of foods, such as 'frozen', 'bakery', 'produce', etc. This seems like a larger grouping system then either looking at the individual names or the aisles, since a 'frozen' department might have many aisles, one for ice cream, one for frozen diners, etc. So the hierarchy of food groupings that we might use are:

- Individual items (using the product_name)
- Aisle
- Department

In [20]:
orders_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [6]:
orders_prior.join(orders)

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
5,2,17794,6,1
6,2,40141,7,1
7,2,1819,8,1
8,2,43668,9,0
9,3,33754,1,1


In [16]:
orders.set_index('order_id', inplace=True, drop=False)

In [18]:
priors = priors.join(orders, on='order_id', rsuffix='_')

NameError: name 'priors' is not defined

In [ ]:
priors = 

In [22]:
orders.loc[orders['user_id'] == 2]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
order_id,,,,,,,
2168274,2168274,2,prior,1,2,11,NaN
1501582,1501582,2,prior,2,5,10,10.0
1901567,1901567,2,prior,3,1,10,3.0
738281,738281,2,prior,4,2,10,8.0
1673511,1673511,2,prior,5,3,11,8.0
1199898,1199898,2,prior,6,2,9,13.0
3194192,3194192,2,prior,7,2,12,14.0
788338,788338,2,prior,8,1,15,27.0
1718559,1718559,2,prior,9,2,9,8.0


In [ ]:
orders.merge()

In [12]:
orders.loc[orders['eval_set'] == 'train'].head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
10,1187899,1,train,11,4,8,14.0
25,1492625,2,train,15,1,11,30.0
49,2196797,5,train,5,0,11,6.0
74,525192,7,train,21,2,11,6.0
78,880375,8,train,4,1,14,10.0


In [3]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [4]:
products.shape[0]

49688

There are about 50,000 products, each with aisle_id and department_id

In [5]:
departments.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [6]:
departments.shape[0]

21

There are 21 departments

In [7]:
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [8]:
aisles.shape[0]

134

All together we have the following statistics:
    
- products: ~50,000
- departments: 21
- aisles: 134

There are about 134 aisles

## The three 'orders' tables

There are three sets of data which have information about the orders that customers have made.

**Orders**: This table includes information about all the orders placed. The features are:
- *order_id*
- *user_id*
- *eval_set*: this feature tells which of the three sets the order belongs in (prior, train, test)
- *order_number*
- *order_dow*: not sure what this feature is
- *order_hour_of_the_day*: hour of the day the order was places (out of a 24 hour clock)
- *days_since_prior_order*

The `orders` table does not include information about the individual products that go into each order. `orders` records the relationship between a user and the order. 

The tables `orders_prior` and `orders_train` relate the individual products (represented by product ids) to the orders. These two tables also record the order in which the item was added to the order and it this item is a reorder. These tables have the following features:

- *order_id*
- *product_id*
- *add_to_cart_order*: what order the item was added to the cart. Was it the first added? The last?
- *reordered*: Has this person ordered this item in the past?

In order to get a large dataframe which includes all the information about the individual orders and the order itself we would have to concatenate the `orders_prior` and the `orders_train` table and then merge the new table with the orders table on the `order_id` column. We want to include all the orders, even if there are not products in the order so we would left join the `orders_prior + orders_train` on the `orders` table which would look something like this:

```python
df = pd.concat((orders_train, orders_prior), axis=0)
df = orders.merge(df, on='order_id', how='left')
```

Notice that every order can have multiple items associated to it. For example, take order_id 2539329. This is an order labeled as "prior" and associated to user_id 1.

In [9]:
orders_train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [10]:
orders.loc[orders['user_id'] == 1]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


Notice that user 1 has multiple orders. All orders except order 1187899 are marked as prior and order 1187899 is marked as train. Each of these orders can have multiple items associated to it:

In [11]:
orders_prior.loc[orders_prior['order_id'] == 2539329]

,order_id,product_id,add_to_cart_order,reordered
24076664,2539329,196,1,0
24076665,2539329,14084,2,0
24076666,2539329,12427,3,0
24076667,2539329,26088,4,0
24076668,2539329,26405,5,0


In [12]:
orders_train.loc[orders_train['order_id'] == 2539329]

,order_id,product_id,add_to_cart_order,reordered


In [13]:
orders_train.loc[orders_train['order_id'] == 1187899]

,order_id,product_id,add_to_cart_order,reordered
484420,1187899,196,1,1
484421,1187899,25133,2,1
484422,1187899,38928,3,1
484423,1187899,26405,4,1
484424,1187899,39657,5,1
484425,1187899,10258,6,1
484426,1187899,13032,7,1
484427,1187899,26088,8,1
484428,1187899,27845,9,0
484429,1187899,49235,10,1


The prior orders are there to help predict the items in the test set.

Notice that there is no table for orders associated to the train orders because the goal of the exercise is to predict these items that should be in the test orders.

In [14]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [15]:
orders['eval_set'].unique()

array(['prior', 'train', 'test'], dtype=object)

In [16]:
orders_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [17]:
orders_prior.shape[0]

32434489

In [18]:
len(orders_prior['order_id'].unique())

3214874

There are about 32.5 million rows in the prior orders table which represent items being places in orders. THe number of orders is about 3.2 million.

In [19]:
orders_train.shape

(1384617, 4)

In [20]:
orders_train['order_id'].unique()

array([      1,      36,      38, ..., 3421058, 3421063, 3421070])

In [21]:
sample_submission.head()

,order_id,products
0,17,39276 29259
1,34,39276 29259
2,137,39276 29259
3,182,39276 29259
4,257,39276 29259


In [22]:
orders.loc[orders['eval_set'] == 'test'].head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
38,2774568,3,test,13,5,15,11.0
44,329954,4,test,6,3,12,30.0
53,1528013,6,test,4,3,16,22.0
96,1376945,11,test,8,6,11,8.0
102,1356845,12,test,6,1,20,30.0


There are about 1.4 million rows in the training data

In [23]:
orders_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [24]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [4]:
orders_full = pd.read_csv('~/instacart_data/orders_full.csv')

In [ ]:
orders_full.head()

In [ ]:
orders_full = orders_full.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
orders_full.head()

In [6]:
orders_full.loc[orders_full['user_id'] == 2].head()

,Unnamed: 0,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
70,70,2168274,2,prior,1,2,11,NaN,32792.0,1.0,0.0
71,71,2168274,2,prior,1,2,11,NaN,47766.0,2.0,0.0
72,72,2168274,2,prior,1,2,11,NaN,20574.0,3.0,0.0
73,73,2168274,2,prior,1,2,11,NaN,12000.0,4.0,0.0
74,74,2168274,2,prior,1,2,11,NaN,48110.0,5.0,0.0


In [5]:
orders_full.loc[orders_full['eval_set'] == 'test'].head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
384,2774568,3,test,13,5,15,11.0,NaN,NaN,NaN
403,329954,4,test,6,3,12,30.0,NaN,NaN,NaN
464,1528013,6,test,4,3,16,22.0,NaN,NaN,NaN
1086,1376945,11,test,8,6,11,8.0,NaN,NaN,NaN
1161,1356845,12,test,6,1,20,30.0,NaN,NaN,NaN


In [11]:
train_set = pd.read_csv('../data/train_set_small.csv')
validation_set = pd.read_csv('../data/validation_set_small.csv')
test_set = pd.read_csv('../data/test_set_small.csv')

In [12]:
pd.concat([train_set, validation_set, test_set])

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,1913360,587,prior,1,4,9,NaN,27521.0,1.0,0.0
1,1913360,587,prior,1,4,9,NaN,40604.0,2.0,0.0
2,1913360,587,prior,1,4,9,NaN,43076.0,3.0,0.0
3,1913360,587,prior,1,4,9,NaN,30290.0,4.0,0.0
4,1913360,587,prior,1,4,9,NaN,15700.0,5.0,0.0
5,1913360,587,prior,1,4,9,NaN,2732.0,6.0,0.0
6,1913360,587,prior,1,4,9,NaN,6297.0,7.0,0.0
7,868416,587,prior,2,5,10,8.0,46822.0,1.0,0.0
8,868416,587,prior,2,5,10,8.0,44632.0,2.0,0.0
9,166174,587,prior,3,6,10,22.0,15950.0,1.0,0.0


In [13]:
train_set.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,1913360,587,prior,1,4,9,NaN,27521.0,1.0,0.0
1,1913360,587,prior,1,4,9,NaN,40604.0,2.0,0.0
2,1913360,587,prior,1,4,9,NaN,43076.0,3.0,0.0
3,1913360,587,prior,1,4,9,NaN,30290.0,4.0,0.0
4,1913360,587,prior,1,4,9,NaN,15700.0,5.0,0.0


In [21]:
full_df = pd.read_csv('../data/df_full.csv')

In [30]:
full_df.loc[full_df['eval_set'] == 'train']

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
107,3054234,587,train,13,6,8,13.0,27521.0,1.0,1.0
108,3054234,587,train,13,6,8,13.0,49086.0,2.0,1.0
109,3054234,587,train,13,6,8,13.0,18224.0,3.0,1.0
110,3054234,587,train,13,6,8,13.0,44282.0,4.0,0.0
111,3054234,587,train,13,6,8,13.0,5785.0,5.0,1.0
291,510056,5870,train,23,5,20,2.0,8174.0,1.0,1.0
292,510056,5870,train,23,5,20,2.0,42265.0,2.0,1.0
293,510056,5870,train,23,5,20,2.0,20039.0,3.0,1.0
294,510056,5870,train,23,5,20,2.0,40541.0,4.0,0.0
295,510056,5870,train,23,5,20,2.0,41284.0,5.0,0.0


In [33]:
full_df.loc[full_df['eval_set'] == 'train'].shape

(1759, 10)

In [35]:
# first have to run generate_full_orders_df.py
orders_full = pd.read_csv('~/instacart_data/orders_full.csv')

In [37]:
def sample_orders(classification='train', sample_size=100, seed=0):
    mask = orders_full['eval_set'] == classification
    return orders_full[mask]['user_id'].sample(sample_size, random_state=seed).values

# Random sample and make a 70%, 30% split for train, validation, and test
train_set_users = sample_orders('train', 100, 1)
test_set_users = sample_orders('test', 30, 6)


In [56]:
train_set_users

array([185816,  96640,  21497, 167844, 170027, 172690, 205935,  69564,
       182761,  20218,  60852, 128332, 149580, 187580,  74439,  59344,
        23726, 199314, 137438,  73241,  79208, 132445, 142532,  61647,
       128986, 195050, 151756,  99035,  62009,  19021, 180821,  78771,
         9024,  44961,  46076,  37022,  64803, 104912, 143812, 150975,
        26000,  26445, 119324,  41388,  72630,  67126, 111726,  74431,
       140742, 193826, 170184, 187355, 141077, 199713, 161090, 183736,
       176147, 131680,  26161, 132482, 181667,  43563, 119566, 125312,
       197420,  49299,  62750, 175925,  54683,  38431, 147333,    587,
       109062,  23893,  96889, 153509,  67312,   5870, 104186, 148636,
       143459, 114975, 154215,  87912, 156175,  73769,  18950,  67682,
       186299,  54787,  31645,   9855, 145725,  86348,  62873,  28895,
       183529, 189531, 115186,  91797])

In [42]:
# check that none of the train, validation, or test sets intersect
assert not set(train_set_users).intersection(set(test_set_users))

In [43]:
# find all orders in orders_full whose users are in each set
train_set = orders_full.loc[orders_full['user_id'].isin(train_set_users)]
test_set = orders_full.loc[orders_full['user_id'].isin(test_set_users)]

In [60]:
train_set.loc[train_set['eval_set'] == 'train'].shape

(1759, 10)

In [55]:
df_full = pd.concat([train_set, test_set])

In [46]:
df_full.loc[df_full['eval_set'] == 'train'].shape

(1759, 10)